In [1]:
# Import libraries
import os
import pandas as pd

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [4]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/My Drive/MLA4H/project

/content/drive/My Drive/MLA4H/project


# 1. Read Raw CSV file

In [37]:
df = pd.read_csv('../data/raw_Op_Hp_LabEvents.csv')
df.tail(5)

,max_FreeCalcium1,min_FreeCalcium1,avg_FreeCalcium1,max_IonizedCalcium,min_IonizedCalcium,avg_IonizedCalcium,max_TotalCalcium1,min_TotalCalcium1,avg_TotalCalcium1,max_TotalCalcium2,...,max_CalciumUrine,min_CalciumUrine,avg_CalciumUrine,max_ParathyroidHormone,min_ParathyroidHormone,avg_ParathyroidHormone,anchor_age,gender,Osteoporosis,Hyperparathyroidism
290501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,34,M,0,0
290502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,85,F,1,0
290503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,50,F,0,0
290504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,57,M,0,0
290505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,64,F,0,0


# 2. Processing Categorical Data

In [38]:
df['gender'] = df['gender'].map({'M': 1, 'F': 0})

# 3. Replacing NaN Values

In [39]:
print(df.isna().sum())

max_FreeCalcium1          248229
min_FreeCalcium1          248229
avg_FreeCalcium1          248229
max_IonizedCalcium        290506
min_IonizedCalcium        290506
avg_IonizedCalcium        290506
max_TotalCalcium1         290506
min_TotalCalcium1         290506
avg_TotalCalcium1         290506
max_TotalCalcium2         290506
min_TotalCalcium2         290506
avg_TotalCalcium2         290506
max_25OHVitaminD          285592
min_25OHVitaminD          285592
avg_25OHVitaminD          285592
max_FreeCalcium2          290506
min_FreeCalcium2          290506
avg_FreeCalcium2          290506
max_CalciumTotal            2091
min_CalciumTotal            2091
avg_CalciumTotal            2091
max_CalciumUrine          289025
min_CalciumUrine          289025
avg_CalciumUrine          289025
max_ParathyroidHormone    285604
min_ParathyroidHormone    285604
avg_ParathyroidHormone    285604
anchor_age                     0
gender                         0
Osteoporosis                   0
Hyperparat

In [40]:
df.fillna(df.median(), inplace=True)
df.dropna(axis=1, how='all', inplace=True)


In [41]:
df.tail()

,max_FreeCalcium1,min_FreeCalcium1,avg_FreeCalcium1,max_25OHVitaminD,min_25OHVitaminD,avg_25OHVitaminD,max_CalciumTotal,min_CalciumTotal,avg_CalciumTotal,max_CalciumUrine,min_CalciumUrine,avg_CalciumUrine,max_ParathyroidHormone,min_ParathyroidHormone,avg_ParathyroidHormone,anchor_age,gender,Osteoporosis,Hyperparathyroidism
290501,1.17,1.07,1.12,24.0,24.0,24.0,8.5,8.5,8.5,3.4,3.2,3.3,86.0,74.0,81.0,34,1,0,0
290502,1.17,1.07,1.12,24.0,24.0,24.0,9.2,9.2,9.2,3.4,3.2,3.3,86.0,74.0,81.0,85,0,1,0
290503,1.17,1.07,1.12,24.0,24.0,24.0,8.4,8.4,8.4,3.4,3.2,3.3,86.0,74.0,81.0,50,0,0,0
290504,1.17,1.07,1.12,24.0,24.0,24.0,8.5,8.5,8.5,3.4,3.2,3.3,86.0,74.0,81.0,57,1,0,0
290505,1.17,1.07,1.12,24.0,24.0,24.0,8.6,8.6,8.6,3.4,3.2,3.3,86.0,74.0,81.0,64,0,0,0


In [42]:
# Save DataFrame to CSV
df.to_csv('../data/processed_Op_Hp_LabEvents.csv', index=False)